### learning rate = 0.005, used convolution for extracting features

In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# Model Description
model_name = 'model-aw-lex-1-3'
model_dir = '../output/all-word/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

with open('/data/aviraj/dataset/train_val_data_fine/all_word_lex','rb') as f:
    train_data, val_data = pickle.load(f)    
    
# Parameters
mode = 'train'
num_senses = 45
num_pos = 12
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 256
num_filter = 256
kernel_size = 5
keep_prob = 0.3
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 500
decay_rate = 0.99
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 6

In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL
device_num = 0
tower_grads = []
losses = []
predictions = []
predictions_pos = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
y_pos = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
sense_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='sense_mask')
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
pretrain = tf.placeholder('bool', [], name="pretrain")

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)
summaries = []

def global_attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        if gpu_idx>int(num_gpus/2)-1:
            device_num = 1
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % device_num):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), 1)

            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)

            with tf.variable_scope("global_attention"):
                W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*10))
                c = tf.expand_dims(global_attention(h[0], x_mask[gpu_idx][0], W_att), 0)
                for i in range(1, batch_size):
                    c = tf.concat([c, tf.expand_dims(global_attention(h[i], x_mask[gpu_idx][i], W_att), 0)], 0)
                cc = tf.expand_dims(c, 1)
                c_final = tf.tile(cc, [1, max_sent_size, 1])

            rev_bw_h2 = tf.reverse(bw_h2, [1])

            with tf.variable_scope("convolution"):
                conv1_fw = tf.layers.conv1d(inputs=fw_h2, filters=num_filter, kernel_size=[kernel_size], padding='same', activation=tf.nn.relu)
                conv2_fw = tf.layers.conv1d(inputs=conv1_fw, filters=num_filter, kernel_size=[kernel_size], padding='same')
                conv1_bw_rev = tf.layers.conv1d(inputs=rev_bw_h2, filters=num_filter, kernel_size=[kernel_size], padding='same', activation=tf.nn.relu)
                conv2_bw_rev = tf.layers.conv1d(inputs=conv1_bw_rev, filters=num_filter, kernel_size=[kernel_size], padding='same')
                conv2_bw = tf.reverse(conv2_bw_rev, [1])

            h_final = tf.concat([c_final, conv2_fw, conv2_bw], 2)
            flat_h_final = tf.reshape(h_final, [-1, tf.shape(h_final)[2]])

            with tf.variable_scope("hidden_layer"):
                W = tf.get_variable("W", shape=[4*hidden_size, 2*hidden_size], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[2*hidden_size], initializer=tf.zeros_initializer())
                drop_flat_h_final = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_hl = tf.matmul(drop_flat_h_final, W) + b

            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_sense = tf.matmul(drop_flat_hl, W) + b
                logits = tf.reshape(flat_logits_sense, [batch_size, max_sent_size, num_senses])
                predictions.append(tf.arg_max(logits, 2))

            with tf.variable_scope("softmax_layer_pos"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_pos], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*30))
                b = tf.get_variable("b", shape=[num_pos], initializer=tf.zeros_initializer())
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_pos = tf.matmul(drop_flat_hl, W) + b
                logits_pos = tf.reshape(flat_logits_pos, [batch_size, max_sent_size, num_pos])
                predictions_pos.append(tf.arg_max(logits_pos, 2))


            float_sense_mask = tf.cast(sense_mask[gpu_idx], 'float')
            float_x_mask = tf.cast(x_mask[gpu_idx], 'float')

            loss = tf.contrib.seq2seq.sequence_loss(logits, y[gpu_idx], float_sense_mask, name="loss")
            loss_pos = tf.contrib.seq2seq.sequence_loss(logits_pos, y_pos[gpu_idx], float_x_mask, name="loss_")

            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = tf.cond(pretrain, lambda:loss_pos, lambda:loss + loss_pos + l2_loss)

            summaries.append(tf.summary.scalar("loss_{}".format(gpu_idx), loss))
            summaries.append(tf.summary.scalar("loss_pos_{}".format(gpu_idx), loss_pos))
            summaries.append(tf.summary.scalar("total_loss_{}".format(gpu_idx), total_loss))


            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

tower_grads = average_gradients(tower_grads)
losses = tf.add_n(losses)/len(losses)
apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
summaries.append(tf.summary.scalar('total_loss', losses))
summaries.append(tf.summary.scalar('learning_rate', learning_rate))

for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))

variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())

train_op = tf.group(apply_grad_op, variables_averages_op)
saver = tf.train.Saver(tf.global_variables())
summary = tf.summary.merge(summaries)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
save_period = 100
log_period = 100

def model(xx, yy, yy_pos, mask, smask, train_cond=True, pretrain_cond=False):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    temp_loss = 0
    preds_sense = []
    true_sense = []
    preds_pos = []
    true_pos = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size, -1])
        yy_pos_re = yy_pos[s:e].reshape([num_gpus, batch_size, -1])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        smask_re = smask[s:e].reshape([num_gpus, batch_size, -1])
 
        feed_dict = {x:xx_re, y:yy_re, y_pos:yy_pos_re, x_mask:mask_re, sense_mask:smask_re, pretrain:pretrain_cond, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            
            temp_loss += _loss
            if((j+1)%log_period==0):
                print("Steps: {}".format(step), "Loss:{0:.4f}".format(temp_loss/log_period), ", Current Loss: {0:.4f}".format(_loss))
                temp_loss = 0
            if((j+1)%save_period==0):
                saver.save(sess, save_path=save_dir)                         
                
        else:
            _loss, pred, pred_pos = sess.run([total_loss, predictions, predictions_pos], feed_dict)
            for i in range(num_gpus):
                preds_sense.append(pred[i][smask_re[i]])
                true_sense.append(yy_re[i][smask_re[i]])
                preds_pos.append(pred_pos[i][mask_re[i]])
                true_pos.append(yy_pos_re[i][mask_re[i]])

        _losses +=_loss

    if(train_cond==False): 
        sense_preds = []
        sense_true = []
        pos_preds = []
        pos_true = []
        
        for preds in preds_sense:
            for ps in preds:      
                sense_preds.append(ps)  
        for trues in true_sense:
            for ts in trues:
                sense_true.append(ts)
        
        for preds in preds_pos:
            for ps in preds:      
                pos_preds.append(ps)      
        for trues in true_pos:
            for ts in trues:
                pos_true.append(ts)
                
        return _losses/num_batches, sense_preds, sense_true, pos_preds, pos_true

    return _losses/num_batches, step

def eval_score(yy, pred, yy_pos, pred_pos):
    f1 = f1_score(yy, pred, average='macro')
    accu = accuracy_score(yy, pred)
    f1_pos = f1_score(yy_pos, pred_pos, average='macro')
    accu_pos = accuracy_score(yy_pos, pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

In [7]:
x_id_train = train_data['x']
mask_train = train_data['x_mask']
sense_mask_train = train_data['sense_mask']
y_train = train_data['y']
y_pos_train = train_data['pos']

x_id_val = val_data['x']
mask_val = val_data['x_mask']
sense_mask_val = val_data['sense_mask']
y_val = val_data['y']
y_pos_val = val_data['pos']

In [8]:
num_epochs = 1
val_period = 1
loss_collection = []
val_collection = []
pre_train_cond = True
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 100 Loss:33.2750 , Current Loss: 0.8327
Steps: 200 Loss:0.5268 , Current Loss: 0.3678
Steps: 300 Loss:0.3184 , Current Loss: 0.2711
Steps: 400 Loss:0.2548 , Current Loss: 0.2264
Steps: 500 Loss:0.2194 , Current Loss: 0.2027
Steps: 600 Loss:0.1966 , Current Loss: 0.1826
Steps: 700 Loss:0.1808 , Current Loss: 0.1779
Steps: 800 Loss:0.1699 , Current Loss: 0.1736
Steps: 900 Loss:0.1617 , Current Loss: 0.1602
Steps: 1000 Loss:0.1529 , Current Loss: 0.1488
Steps: 1100 Loss:0.1489 , Current Loss: 0.1545
Steps: 1200 Loss:0.1458 , Current Loss: 0.1462
Steps: 1300 Loss:0.1404 , Current Loss: 0.1401
Steps: 1400 Loss:0.1356 , Current Loss: 0.1399
Steps: 1500 Loss:0.1314 , Current Loss: 0.1258
Steps: 1600 Loss:0.1298 , Current Loss: 0.1300
Steps: 1700 Loss:0.1274 , Current Loss: 0.1163
Epoch: 1 , Step: 1771 , loss: 2.0612 , Time: 11504.6
Model Saved


/users/btech/aviraj/envs/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 Score:1.12 Accuracy:1.47  POS: F1 Score:93.27 Accuracy:95.85 Loss:11.7837 , Time: 2299.4


In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 1871 Loss:3.9956 , Current Loss: 3.1696
Steps: 1971 Loss:2.9681 , Current Loss: 2.8126
Steps: 2071 Loss:2.7876 , Current Loss: 2.7860
Steps: 2171 Loss:2.6289 , Current Loss: 2.5738
Steps: 2271 Loss:2.5435 , Current Loss: 2.3657
Steps: 2371 Loss:2.4744 , Current Loss: 2.5109
Steps: 2471 Loss:2.4250 , Current Loss: 2.4612
Steps: 2571 Loss:2.3814 , Current Loss: 2.3583
Steps: 2671 Loss:2.3216 , Current Loss: 2.3235
Steps: 2771 Loss:2.2793 , Current Loss: 2.3103
Steps: 2871 Loss:2.2659 , Current Loss: 2.2289
Steps: 2971 Loss:2.2372 , Current Loss: 2.2791
Steps: 3071 Loss:2.2210 , Current Loss: 2.1201
Steps: 3171 Loss:2.1979 , Current Loss: 2.1856


In [12]:
for v in val_collection:
    print("Val: F1 Score:{0:.2f}".format(v[0]), "Accuracy:{0:.2f}".format(v[1]), " POS: F1 Score:{0:.2f}".format(v[2]), "Accuracy:{0:.2f}".format(v[3]))

Val: F1 Score:34.22 Accuracy:48.23  POS: F1 Score:83.57 Accuracy:92.12
Val: F1 Score:42.38 Accuracy:53.07  POS: F1 Score:84.18 Accuracy:92.44
Val: F1 Score:44.58 Accuracy:54.93  POS: F1 Score:84.83 Accuracy:92.85
Val: F1 Score:46.84 Accuracy:56.83  POS: F1 Score:84.66 Accuracy:92.92
Val: F1 Score:48.40 Accuracy:57.32  POS: F1 Score:84.70 Accuracy:92.87


In [14]:
loss_collection

[2.4655171245723575,
 2.0027206975278848,
 1.8724895704943605,
 1.7916589952471027,
 1.736836715716551,
 1.6930595723200086,
 1.6597260672247793,
 1.6344640272475446,
 1.6094474871908453,
 1.594043997436644]

In [9]:
sess.run(learning_ratening_ratening_rate)

0.00099700305

In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 19581 Loss:1.5124 , Current Loss: 1.4568
Steps: 19681 Loss:1.4886 , Current Loss: 1.5131
Steps: 19781 Loss:1.4710 , Current Loss: 1.5145
Steps: 19881 Loss:1.4741 , Current Loss: 1.4782
Steps: 19981 Loss:1.4605 , Current Loss: 1.4133
Steps: 20081 Loss:1.4476 , Current Loss: 1.5403
Steps: 20181 Loss:1.4435 , Current Loss: 1.4998
Steps: 20281 Loss:1.4296 , Current Loss: 1.3782
Steps: 20381 Loss:1.4368 , Current Loss: 1.4933
Steps: 20481 Loss:1.4205 , Current Loss: 1.3596
Steps: 20581 Loss:1.4142 , Current Loss: 1.4263
Steps: 20681 Loss:1.4230 , Current Loss: 1.4139
Steps: 20781 Loss:1.4181 , Current Loss: 1.4190
Steps: 20881 Loss:1.4126 , Current Loss: 1.3405
Steps: 20981 Loss:1.4132 , Current Loss: 1.3976
Steps: 21081 Loss:1.4086 , Current Loss: 1.3606
Steps: 21181 Loss:1.3866 , Current Loss: 1.4127
Epoch: 1 , Step: 21252 , loss: 1.4373 , Time: 11614.5
Model Saved
Steps: 21352 Loss:1.3863 , Current Loss: 1.4108
Steps: 21452 Loss:1.3959 , Current Loss: 1.3552
Steps: 21552 Loss:1.39

In [ ]:
start_time = time.time()
val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
time_taken = time.time() - start_time
print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Val: F1 Score:55.43 Accuracy:63.29  POS: F1 Score:86.72 Accuracy:94.23 Loss:1.3318 , Time: 2293.5


In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 26665 Loss:1.3179 , Current Loss: 1.2862
Steps: 26765 Loss:1.3207 , Current Loss: 1.2462
Steps: 26865 Loss:1.3363 , Current Loss: 1.2352
Steps: 26965 Loss:1.3255 , Current Loss: 1.3320
Steps: 27065 Loss:1.3253 , Current Loss: 1.3032
Steps: 27165 Loss:1.3140 , Current Loss: 1.3353
Steps: 27265 Loss:1.3248 , Current Loss: 1.3165
Steps: 27365 Loss:1.3272 , Current Loss: 1.1715
Steps: 27465 Loss:1.3228 , Current Loss: 1.2624
Steps: 27565 Loss:1.3206 , Current Loss: 1.3002
Steps: 27665 Loss:1.3233 , Current Loss: 1.2588
Steps: 27765 Loss:1.3185 , Current Loss: 1.2974
Steps: 27865 Loss:1.3174 , Current Loss: 1.2037
Steps: 27965 Loss:1.3160 , Current Loss: 1.2747
Steps: 28065 Loss:1.3163 , Current Loss: 1.2220
Steps: 28165 Loss:1.3213 , Current Loss: 1.2834
Steps: 28265 Loss:1.3144 , Current Loss: 1.3345
Epoch: 1 , Step: 28336 , loss: 1.3213 , Time: 11358.2
Model Saved
Steps: 28436 Loss:1.3049 , Current Loss: 1.2685
Steps: 28536 Loss:1.3143 , Current Loss: 1.2625
Steps: 28636 Loss:1.33

In [ ]:
start_time = time.time()
val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
time_taken = time.time() - start_time
print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Val: F1 Score:57.40 Accuracy:64.72  POS: F1 Score:87.31 Accuracy:94.46 Loss:1.2828 , Time: 2283.2


In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 37291 Loss:1.2675 , Current Loss: 1.3723
Steps: 37391 Loss:1.2758 , Current Loss: 1.2771
Steps: 37491 Loss:1.2539 , Current Loss: 1.2049
Steps: 37591 Loss:1.2661 , Current Loss: 1.2917
Steps: 37691 Loss:1.2715 , Current Loss: 1.2075
Steps: 37791 Loss:1.2647 , Current Loss: 1.2314
Steps: 37891 Loss:1.2737 , Current Loss: 1.2460
Steps: 37991 Loss:1.2699 , Current Loss: 1.2344
Steps: 38091 Loss:1.2749 , Current Loss: 1.1636
Steps: 38191 Loss:1.2595 , Current Loss: 1.2154
Steps: 38291 Loss:1.2720 , Current Loss: 1.3131
Steps: 38391 Loss:1.2749 , Current Loss: 1.3101
Steps: 38491 Loss:1.2565 , Current Loss: 1.2913
Steps: 38591 Loss:1.2671 , Current Loss: 1.3980
Steps: 38691 Loss:1.2660 , Current Loss: 1.3388
Steps: 38791 Loss:1.2650 , Current Loss: 1.3399
Steps: 38891 Loss:1.2570 , Current Loss: 1.3982
Epoch: 1 , Step: 38962 , loss: 1.2666 , Time: 11799.0
Model Saved
Steps: 39062 Loss:1.2543 , Current Loss: 1.2514
Steps: 39162 Loss:1.2598 , Current Loss: 1.3367
Steps: 39262 Loss:1.25

In [9]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 47917 Loss:1.2285 , Current Loss: 1.1608
Steps: 48017 Loss:1.2170 , Current Loss: 1.2742
Steps: 48117 Loss:1.2296 , Current Loss: 1.2536
Steps: 48217 Loss:1.2181 , Current Loss: 1.2226
Steps: 48317 Loss:1.2240 , Current Loss: 1.2340
Steps: 48417 Loss:1.2253 , Current Loss: 1.1124
Steps: 48517 Loss:1.2299 , Current Loss: 1.1854
Steps: 48617 Loss:1.2215 , Current Loss: 1.2742
Steps: 48717 Loss:1.2125 , Current Loss: 1.2555
Steps: 48817 Loss:1.2219 , Current Loss: 1.3055
Steps: 48917 Loss:1.2280 , Current Loss: 1.3099
Steps: 49017 Loss:1.2145 , Current Loss: 1.1509
Steps: 49117 Loss:1.2237 , Current Loss: 1.2367
Steps: 49217 Loss:1.2122 , Current Loss: 1.2078
Steps: 49317 Loss:1.2217 , Current Loss: 1.1881
Steps: 49417 Loss:1.2199 , Current Loss: 1.1954
Steps: 49517 Loss:1.2260 , Current Loss: 1.1849
Epoch: 1 , Step: 49588 , loss: 1.2224 , Time: 12097.6
Model Saved
Steps: 49688 Loss:1.2229 , Current Loss: 1.2027
Steps: 49788 Loss:1.2132 , Current Loss: 1.3340
Steps: 49888 Loss:1.21

Steps: 63685 Loss:1.1847 , Current Loss: 1.1713
Epoch: 9 , Step: 63756 , loss: 1.1787 , Time: 12027.2
Model Saved
Steps: 63856 Loss:1.1841 , Current Loss: 1.2301
Steps: 63956 Loss:1.1785 , Current Loss: 1.1185
Steps: 64056 Loss:1.1725 , Current Loss: 1.1739
Steps: 64156 Loss:1.1750 , Current Loss: 1.1440
Steps: 64256 Loss:1.1876 , Current Loss: 1.0063
Steps: 64356 Loss:1.1794 , Current Loss: 1.1291
Steps: 64456 Loss:1.1763 , Current Loss: 1.1623
Steps: 64556 Loss:1.1606 , Current Loss: 1.2084
Steps: 64656 Loss:1.1800 , Current Loss: 1.2594
Steps: 64756 Loss:1.1724 , Current Loss: 1.1070
Steps: 64856 Loss:1.1818 , Current Loss: 1.1023
Steps: 64956 Loss:1.1778 , Current Loss: 1.2261
Steps: 65056 Loss:1.1746 , Current Loss: 1.2680
Steps: 65156 Loss:1.1798 , Current Loss: 1.0067
Steps: 65256 Loss:1.1757 , Current Loss: 1.2404
Steps: 65356 Loss:1.1727 , Current Loss: 1.0650
Steps: 65456 Loss:1.1790 , Current Loss: 1.1002
Epoch: 10 , Step: 65527 , loss: 1.1771 , Time: 12191.0
Model Saved
Val

In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 65627 Loss:1.1703 , Current Loss: 1.1085
Steps: 65727 Loss:1.1604 , Current Loss: 1.1488
Steps: 65827 Loss:1.1683 , Current Loss: 1.1013
Steps: 65927 Loss:1.1680 , Current Loss: 1.0642
Steps: 66027 Loss:1.1671 , Current Loss: 1.0811
Steps: 66127 Loss:1.1771 , Current Loss: 1.1177
Steps: 66227 Loss:1.1537 , Current Loss: 1.0352
Steps: 66327 Loss:1.1576 , Current Loss: 1.0934
Steps: 66427 Loss:1.1563 , Current Loss: 1.0517
Steps: 66527 Loss:1.1571 , Current Loss: 1.0961
Steps: 66627 Loss:1.1623 , Current Loss: 1.1719
Steps: 66727 Loss:1.1747 , Current Loss: 1.1284
Steps: 66827 Loss:1.1650 , Current Loss: 1.0858
Steps: 66927 Loss:1.1776 , Current Loss: 1.0603
Steps: 67027 Loss:1.1716 , Current Loss: 1.1617
Steps: 67127 Loss:1.1589 , Current Loss: 1.1192
Steps: 67227 Loss:1.1554 , Current Loss: 1.1729
Epoch: 1 , Step: 67298 , loss: 1.1650 , Time: 13973.3
Model Saved
Steps: 67398 Loss:1.1583 , Current Loss: 1.1437
Steps: 67498 Loss:1.1657 , Current Loss: 1.0963
Steps: 67598 Loss:1.15

In [9]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append(train_loss)
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 79795 Loss:1.1200 , Current Loss: 1.0775
Steps: 79895 Loss:1.1391 , Current Loss: 1.1888
Steps: 79995 Loss:1.1328 , Current Loss: 1.0539
Steps: 80095 Loss:1.1264 , Current Loss: 0.9866
Steps: 80195 Loss:1.1351 , Current Loss: 1.0090
Steps: 80295 Loss:1.1246 , Current Loss: 1.0305
Steps: 80395 Loss:1.1291 , Current Loss: 1.0609
Steps: 80495 Loss:1.1463 , Current Loss: 1.1761
Steps: 80595 Loss:1.1315 , Current Loss: 1.0556
Steps: 80695 Loss:1.1244 , Current Loss: 1.2292
Steps: 80795 Loss:1.1337 , Current Loss: 1.1745
Steps: 80895 Loss:1.1125 , Current Loss: 0.9700
Steps: 80995 Loss:1.1326 , Current Loss: 1.0512
Steps: 81095 Loss:1.1278 , Current Loss: 1.1351
Steps: 81195 Loss:1.1307 , Current Loss: 1.1419
Steps: 81295 Loss:1.1348 , Current Loss: 1.2109
Steps: 81395 Loss:1.1157 , Current Loss: 1.0764
Epoch: 1 , Step: 81466 , loss: 1.1287 , Time: 12084.3
Model Saved
Steps: 81566 Loss:1.1190 , Current Loss: 1.1888
Steps: 81666 Loss:1.1354 , Current Loss: 1.2150
Steps: 81766 Loss:1.12

Steps: 95563 Loss:1.0942 , Current Loss: 1.2293
Epoch: 9 , Step: 95634 , loss: 1.1024 , Time: 11893.5
Model Saved
Steps: 95734 Loss:1.0971 , Current Loss: 1.0361
Steps: 95834 Loss:1.1053 , Current Loss: 1.1541
Steps: 95934 Loss:1.1053 , Current Loss: 1.0345
Steps: 96034 Loss:1.0991 , Current Loss: 1.0116
Steps: 96134 Loss:1.1110 , Current Loss: 0.9568
Steps: 96234 Loss:1.1031 , Current Loss: 1.1285
Steps: 96334 Loss:1.0941 , Current Loss: 1.0702
Steps: 96434 Loss:1.1066 , Current Loss: 1.0791
Steps: 96534 Loss:1.1142 , Current Loss: 1.2335
Steps: 96634 Loss:1.1107 , Current Loss: 1.0056
Steps: 96734 Loss:1.1169 , Current Loss: 1.0617
Steps: 96834 Loss:1.0999 , Current Loss: 1.0654
Steps: 96934 Loss:1.0870 , Current Loss: 1.0647
Steps: 97034 Loss:1.0903 , Current Loss: 1.1249
Steps: 97134 Loss:1.0934 , Current Loss: 1.1321
Steps: 97234 Loss:1.0999 , Current Loss: 1.2068
Steps: 97334 Loss:1.0973 , Current Loss: 1.1554
Epoch: 10 , Step: 97405 , loss: 1.1017 , Time: 11877.5
Model Saved
Val

In [11]:
for v in val_collection:
    print("Val: F1 Score:{0:.2f}".format(v[0]), "Accuracy:{0:.2f}".format(v[1]), " POS: F1 Score:{0:.2f}".format(v[2]), "Accuracy:{0:.2f}".format(v[3]))

Val: F1 Score:62.32 Accuracy:68.26  POS: F1 Score:88.83 Accuracy:95.01
Val: F1 Score:62.22 Accuracy:68.34  POS: F1 Score:88.68 Accuracy:95.01
Val: F1 Score:62.27 Accuracy:68.44  POS: F1 Score:88.74 Accuracy:95.07
Val: F1 Score:62.40 Accuracy:68.63  POS: F1 Score:88.96 Accuracy:95.08
Val: F1 Score:62.96 Accuracy:68.92  POS: F1 Score:89.02 Accuracy:95.12


In [12]:
loss_collection

[1.0952356004701504, 1.0919322563321627, 1.0906437579889474]

In [11]:
for v in val_collection:
    print("Val: F1 Score:{0:.2f}".format(v[0]), "Accuracy:{0:.2f}".format(v[1]), " POS: F1 Score:{0:.2f}".format(v[2]), "Accuracy:{0:.2f}".format(v[3]))

Val: F1 Score:62.59 Accuracy:68.87  POS: F1 Score:88.98 Accuracy:95.12


In [ ]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append([step, train_loss])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 103519 Loss:1.0857 , Current Loss: 1.0955
Steps: 103619 Loss:1.0943 , Current Loss: 1.0948
Steps: 103719 Loss:1.0899 , Current Loss: 0.8898
Steps: 103819 Loss:1.0767 , Current Loss: 1.0710
Steps: 103919 Loss:1.1007 , Current Loss: 1.0328
Steps: 104019 Loss:1.0872 , Current Loss: 1.0722
Steps: 104119 Loss:1.0919 , Current Loss: 1.1361
Steps: 104219 Loss:1.0784 , Current Loss: 1.0826
Steps: 104319 Loss:1.0812 , Current Loss: 1.0122
Steps: 104419 Loss:1.0748 , Current Loss: 1.1719
Steps: 104519 Loss:1.0790 , Current Loss: 0.9485
Steps: 104619 Loss:1.0880 , Current Loss: 1.0874
Steps: 104719 Loss:1.0905 , Current Loss: 0.9633
Steps: 104819 Loss:1.1007 , Current Loss: 1.0679
Steps: 104919 Loss:1.0879 , Current Loss: 0.9702
Steps: 105019 Loss:1.0761 , Current Loss: 1.1684
Steps: 105119 Loss:1.0782 , Current Loss: 1.1231
Epoch: 1 , Step: 105190 , loss: 1.0864 , Time: 16818.0
Model Saved
Steps: 105290 Loss:1.0955 , Current Loss: 1.2707
Steps: 105390 Loss:1.0776 , Current Loss: 1.1164
St

In [9]:
num_epochs = 10
val_period = 2
loss_collection = []
val_collection = []
pre_train_cond = False
for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    loss_collection.append([step, train_loss])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        val_collection.append([f1_, accu_, f1_pos_, accu_pos_])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 109332 Loss:1.7847 , Current Loss: 1.7501
Steps: 109432 Loss:1.7497 , Current Loss: 1.6552
Steps: 109532 Loss:1.7298 , Current Loss: 1.8444
Steps: 109632 Loss:1.7266 , Current Loss: 1.7739
Steps: 109732 Loss:1.7245 , Current Loss: 1.8321
Steps: 109832 Loss:1.7206 , Current Loss: 1.6995
Steps: 109932 Loss:1.7060 , Current Loss: 1.5771
Steps: 110032 Loss:1.7105 , Current Loss: 1.6508
Steps: 110132 Loss:1.7066 , Current Loss: 1.6644
Steps: 110232 Loss:1.6970 , Current Loss: 1.5904
Steps: 110332 Loss:1.7139 , Current Loss: 1.8116
Steps: 110432 Loss:1.6883 , Current Loss: 1.7113
Steps: 110532 Loss:1.7016 , Current Loss: 1.7064
Steps: 110632 Loss:1.7041 , Current Loss: 1.6901
Steps: 110732 Loss:1.7024 , Current Loss: 1.5544
Steps: 110832 Loss:1.7028 , Current Loss: 1.7255
Steps: 110932 Loss:1.6965 , Current Loss: 1.5244
Epoch: 1 , Step: 111003 , loss: 1.7145 , Time: 11848.3
Model Saved
Steps: 111103 Loss:1.6831 , Current Loss: 1.8166
Steps: 111203 Loss:1.6968 , Current Loss: 1.7384
St

ResourceExhaustedError: OOM when allocating tensor with shape[64,200,512]
	 [[Node: model_0/gradients/model_0/global_attention/strided_slice_grad/StridedSliceGrad = StridedSliceGrad[Index=DT_INT32, T=DT_FLOAT, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1, _device="/job:localhost/replica:0/task:0/gpu:0"](model_0/gradients/model_0/global_attention/strided_slice_grad/Shape, model_0/global_attention/strided_slice/stack, model_0/global_attention/strided_slice/stack_1, model_0/global_attention/strided_slice/stack_2, model_0/gradients/model_0/global_attention/boolean_mask/Reshape_grad/Reshape)]]
	 [[Node: Adam/update/_9206 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_88100_Adam/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'model_0/gradients/model_0/global_attention/strid

In [ ]:
start_time = time.time()
train_loss, train_pred, train_true, train_pred_pos, train_true_pos = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = etrain_score(train_true, train_pred, train_true_pos, train_pred_pos)
time_taken = time.time() - start_time
print("train: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))

In [8]:
saver.restore(sess, save_dir)